# attrs

* Create classes with less boilerplate and less chance for error
* Implement commonly used functionality quickly
* Only used to create classes - then gets out of the way
* Avoid the need for some tedious unit tests

## Boring example

In [1]:
import attr

@attr.s
class Foo:
    x = attr.ib()
    y = attr.ib()  


In [2]:
f = Foo(2, "hi")
print(f)        # nice repr
print(f.x, f.y) # field access

Foo(x=2, y='hi')
2 hi


In [3]:
Foo()   # values must be provided

TypeError: __init__() missing 2 required positional arguments: 'x' and 'y'

In [4]:
Foo(y=2, x=1)  # initialise by keyword arg (order doesn't matter)

Foo(x=1, y=2)

## Defaults

In [9]:
@attr.s
class Bar:
    x = attr.ib(default=42)
    y = attr.ib(factory=list)  
    


In [8]:
class Bar2:
    def __init__(self, x=[]):
        self.x = x
        
b = Bar2()
b.x.append(42)

c = Bar2()
c.x
c.x.append(99)
c.x

b.x

[42, 99]

In [10]:
Bar()

Bar(x=42, y=[])

In [11]:
Bar(9)

Bar(x=9, y=[])

In [12]:
Bar(y=[1,2,3])

Bar(x=42, y=[1, 2, 3])

In [13]:
Bar(y=99)  # default does NOT enforce type

Bar(x=42, y=99)

## Comparison

The magic methods for comparison get automatically implemented by attrs.

In [14]:
@attr.s
class A:
    x = attr.ib()
    y = attr.ib()

In [15]:
A(1, 2) == A(1, 2)  

True

In [16]:
A(1, 2) == A(3, 4)

False

In [17]:
A(2, 3) < A(2, 4)

True

In [18]:
A(5, 1) > A(4, 2)

True

In [19]:
A(1, 2) is A(1, 2)

False

In [20]:
# Different class with the same attributes does not compare.

@attr.s
class B:
    x = attr.ib()
    y = attr.ib()
    
A(1, 2) == B(1, 2)

False

## Differences to namedtuples

In [21]:
# Not iterable
from collections import namedtuple

FooTuple = namedtuple("FooTuple", ["x", "y"])
for z in FooTuple(1, 2):
    print(z)

print("---")

for z in Foo(1, 2):
    print(z)


1
2
---


TypeError: 'Foo' object is not iterable

In [22]:
# No comparison gotcha

FooTuple = namedtuple("FooTuple", ["x", "y"])
BarTuple = namedtuple("BarTuple", ["x", "y"])

FooTuple(3, 4) == BarTuple(3, 4)

True

In [23]:
FooTuple(4, 4) == (4, 4)

True

In [24]:
# No extra method or attribute baggage
[a for a in dir(FooTuple(1, 2)) if not a.startswith("__")]

['_asdict',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'count',
 'index',
 'x',
 'y']

In [25]:
[a for a in dir(Foo(1, 2)) if not a.startswith("__")]

['x', 'y']

## Equivalent Handwritten Class

In [ ]:
@attr.s
class SmartClass(object):
    a = attr.ib()
    b = attr.ib()

is roughly:

In [ ]:
class ArtisanalClass(object):
     def __init__(self, a, b):
         self.a = a
         self.b = b

     def __repr__(self):
         return "ArtisanalClass(a={}, b={})".format(self.a, self.b)

     def __eq__(self, other):
         if other.__class__ is self.__class__:
             return (self.a, self.b) == (other.a, other.b)
         else:
             return NotImplemented

     def __ne__(self, other):
         result = self.__eq__(other)
         if result is NotImplemented:
             return NotImplemented
         else:
             return not result

     def __lt__(self, other):
         if other.__class__ is self.__class__:
             return (self.a, self.b) < (other.a, other.b)
         else:
             return NotImplemented

     def __le__(self, other):
         if other.__class__ is self.__class__:
             return (self.a, self.b) <= (other.a, other.b)
         else:
             return NotImplemented

     def __gt__(self, other):
         if other.__class__ is self.__class__:
             return (self.a, self.b) > (other.a, other.b)
         else:
             return NotImplemented

     def __ge__(self, other):
         if other.__class__ is self.__class__:
             return (self.a, self.b) >= (other.a, other.b)
         else:
             return NotImplemented

     def __hash__(self):
         return hash((self.__class__, self.a, self.b))

## Overriding Methods

In [27]:
@attr.s(cmp=False)
class Thing:
    name = attr.ib()
    species = attr.ib()
    
    def __eq__(self, other):
        if other.__class__ is self.__class__:
             return self.name.lower() == other.name.lower()
        else:
             return NotImplemented

In [28]:
Thing("bob", "frog") == Thing("BOB", "toad")

True

In [29]:
@attr.s(repr=False)
class Other:
    name = attr.ib()
    
    def __repr__(self):
        return "<<{}>>".format(self.name.upper())

Other("amelia")    

<<AMELIA>>

## Slots
Using slots locks down attributes, reduces memory consumption and speeds up attribute access. With attrs it's easy to switch a class to use slots.

In [ ]:
# Instead of...
class Slotted:
    __slots__ = ("x", "y")
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    ...

In [30]:
@attr.s(slots=True)
class Slotted:
    x = attr.ib()
    y = attr.ib()

In [31]:
s = Slotted(2, 3)
s

Slotted(x=2, y=3)

In [32]:
s.z = 99

AttributeError: 'Slotted' object has no attribute 'z'

## Converting to other types

In [33]:
f = Foo(44, 55)
print(attr.astuple(f))
print(attr.asdict(f))

(44, 55)
{'x': 44, 'y': 55}


In [34]:
print(attr.asdict(f, filter=lambda attr, _: attr.name != "y"))

{'x': 44}


## Forcing parameters to be set by keyword only
Python 3 only

In [35]:
@attr.s(kw_only=True)
class Readable:
    x = attr.ib()
    y = attr.ib()
    
Readable(x=4, y=5)

Readable(x=4, y=5)

In [36]:
Readable(4, 5)

TypeError: __init__() takes 1 positional argument but 3 were given

## Immutability

In [39]:
@attr.s(frozen=True)
class Elsa:
    x = attr.ib()
    
e = Elsa(2)
e

Elsa(x=2)

In [40]:
e.x = 9

FrozenInstanceError: 

## Validation

In [41]:
@attr.s
class Checked:
    x = attr.ib(validator=attr.validators.instance_of(int))
    y = attr.ib(validator=attr.validators.in_(["moo", "baa", "grr"]))

In [42]:
Checked(2, "moo")

Checked(x=2, y='moo')

In [43]:
Checked("x", "moo")

TypeError: ("'x' must be <class 'int'> (got 'x' that is a <class 'str'>).", Attribute(name='x', default=NOTHING, validator=<instance_of validator for type <class 'int'>>, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False), <class 'int'>, 'x')

In [44]:
Checked(4, "wat")

ValueError: 'y' must be in ['moo', 'baa', 'grr'] (got 'wat')

alternatively...

In [45]:
@attr.s
class Checked:
    x = attr.ib()
    
    @x.validator
    def check(self, attribute, value):
        if value > 42:
            raise ValueError("x must be smaller or equal to 42")

Checked(10)

Checked(x=10)

In [46]:
Checked(43)

ValueError: x must be smaller or equal to 42

or just arbitrary callables...

In [47]:
def x_smaller_than_y(instance, attribute, value):
    if value >= instance.y:
        raise ValueError("'x' has to be smaller than 'y'!")

@attr.s
class C(object):
    x = attr.ib(validator=[attr.validators.instance_of(int),
                           x_smaller_than_y])
    y = attr.ib()

C(x=3, y=4)


C(x=3, y=4)

In [48]:
C(x=4, y=3)

ValueError: 'x' has to be smaller than 'y'!

## Reflection

In [49]:
attr.fields(Foo)

(Attribute(name='x', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False),
 Attribute(name='y', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False))

## Metadata
Not used by attrs itself but can be used to enrich classes for 3rd party libraries (e.g. serialisation)

In [50]:
@attr.s
class M(object):
    x = attr.ib(metadata={"json.name": "T"})
    y = attr.ib()
    
attr.fields(M)

(Attribute(name='x', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({'json.name': 'T'}), type=None, converter=None, kw_only=False),
 Attribute(name='y', default=NOTHING, validator=None, repr=True, cmp=True, hash=None, init=True, metadata=mappingproxy({}), type=None, converter=None, kw_only=False))

In [53]:
class Abc(namedtuple("_Abc", ['x', 'y'])):
    
    def add(self):
        return self.x + self.y
    
    
x = Abc(2, 3)
x.add()


5

# Python 3 Data Classes
Some of the ideas from attrs are now in the standard library....